# Miscellaneous ERDDAP functionality

In [ ]:
from erddapy import ERDDAP
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import types
import requests

### Get metadata

To see detailed metadata from a dataset, we request the **ncCF** format

In [ ]:
e = ERDDAP(
    server="https://erddap.observations.voiceoftheocean.org/erddap",
    protocol="tabledap")

In [ ]:
dataset_id = "nrt_SEA068_M27"
e.dataset_id = dataset_id
meta = e.to_ncCF()

In [ ]:
attrs = {}
for key_name in dir(meta):
    if key_name[0] == "_":
        continue
    val =  meta.__getattribute__(key_name)
    if type(val) is types.MethodType:
        continue
    attrs[key_name] = val

### Display vector data

Using some HF Radar from coastwatch

https://coastwatch.pfeg.noaa.gov/erddap/griddap/ucsdHfrW1.html

In [ ]:
coastwatch_e = ERDDAP(
    server="https://coastwatch.pfeg.noaa.gov/erddap",
    protocol="griddap",
)
coastwatch_e.dataset_id='ucsdHfrW1'
coastwatch_e.griddap_initialize() 

In [ ]:
coastwatch_e.constraints

Coarsen by 10'000 and make a quick plot

In [ ]:
coastwatch_e['longitude_step']: 100
coastwatch_e['latitude_step']: 100
ds = coastwatch_e.to_xarray()
ds.water_u.plot()

Constrain the data to a smaller region for plotting

In [ ]:
coastwatch_e.constraints = {'time>=': '2023-10-17T12:00:00Z',
 'time<=': '2023-10-17T12:00:00Z',
 'time_step': 1,
 'latitude>=':  48.9,
 'latitude<=':  49.35,
 'latitude_step': 1,                            
 'longitude>=': -123.6,
 'longitude<=': -123,
 'longitude_step': 1}
ds = coastwatch_e.to_xarray()
ds.water_u.plot()

Use [matplotlib quiver](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.quiver.html) to plot surface velocity vectors

In [ ]:
fig = plt.figure(figsize=[8, 8])

ax = fig.add_subplot(projection=ccrs.AzimuthalEquidistant(central_latitude=49, central_longitude=-123))

q = ax.quiver(ds.longitude, ds.latitude, ds.water_u.values[0, :,:],
          ds.water_v.values[0, :, :], scale=20, transform=ccrs.PlateCarree())
q = ax.quiverkey(q, 0.8, 0.9, 1, r'1 m s$^{-1}$', labelpos='E',
                    transform=ccrs.PlateCarree())
ax.gridlines(color='k', alpha=0.2)

ax.set(title=ds.attrs['title']);

### Display satellite data

Pathfinder SST

https://coastwatch.pfeg.noaa.gov/erddap/griddap/erdPH53sstdmday.graph

In [ ]:
satellite_e = ERDDAP(
    server="https://coastwatch.pfeg.noaa.gov/erddap",
    protocol="griddap",
)
satellite_e.dataset_id='erdPH53sstdmday'
satellite_e.griddap_initialize() 

In [ ]:
print("variables: ", satellite_e.variables)
satellite_e.constraints

In [ ]:
satellite_e.variables = ['sea_surface_temperature']
satellite_e.constraints['latitude_step'] = 100
satellite_e.constraints['longitude_step'] = 100
sat = satellite_e.to_xarray()
sat.sea_surface_temperature.plot(figsize=(10, 6), vmin=-5, vmax=35, cmap="RdBu_r")

### Get data into matlab/R

Using the xarray and pandas objects from the previous notebooks, we can write out our datasets as **.csv** or **.nc** files that can be read by a range of programs

**NOTE FOR FUTURE**

Funtionality to download arbitrary filetypes has been added to erddapy in PR #330 https://github.com/ioos/erddapy/pull/330

This will be available in the next release



In [ ]:
df = e.to_pandas()
df.to_csv(f"{dataset_id}.csv")

There is no matlab client for ERDDAP that I am aware of, but you can use Python to download matlab data:

In [ ]:
# Do all you usual edits here for variables, limits etc.
url = e.get_download_url()
matlab_url = url.replace("html", "mat")
matlab_url

We use requests to write out to **.mat**

In [ ]:
response = requests.get(matlab_url)
with open(f"{dataset_id}.mat", "wb") as fout:
    fout.write(response.content)

In [ ]:
import scipy.io
mat = scipy.io.loadmat(f"{dataset_id}.mat")

### Or use an ERDDAP that serves native .mat files 🤷
http://erddap.oleander.bios.edu:8080/erddap/files/oleanderXbtMatFiles/

See notebook 05 for using R. 

### ERDDAP visitors analysis

See notebooks at https://github.com/callumrollo/website-log-parse/

### Computation and data handling in a cloud environment

The Littlest JupyterHub https://tljh.jupyter.org/en/latest/index.html

- Cohost your dataq with your processing
- Set up accounts for users 

---------------------------
### References

Quiver example modified from https://matplotlib.org/stable/gallery/images_contours_and_fields/quiver_demo.html

Read matlab files into python https://stackoverflow.com/questions/874461/read-mat-files-in-python